In [1]:
from phe import paillier
import json
import numpy as np

In [ ]:
from sys import getsizeof

## CNN model familiarity ##

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 3)
        self.fc1 = nn.Linear(400, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

    def reset_parameters(self, seed=None):
        if seed is not None:
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)

        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()

In [4]:
def dict_torch2list(dic):
    dicc = dict()
    for key, val in dic.items():
        dicc[key] = val.tolist().copy()
    return dicc

def dict_torch2ndarray(dic):
    dicc = dict()
    for key, val in dic.items():
        dicc[key] = val.numpy().copy()
    return dicc

def dict_list2torch(dic):
    dicc = dict()
    for key, val in dic.items():
        dicc[key] = torch.tensor(val, dtype=torch.float32)
    return dicc

def dict_list2ndarray(dic):
    dicc = dict()
    for key, val in dic.items():
        dicc[key] = np.array(val)
    return dicc

def dict_ndarray2list(dic):
    dicc = dict()
    for key, val in dic.items():
        dicc[key] = val.tolist().copy()
    return dicc

In [5]:
cnn = MNIST_CNN()

In [6]:
cnn_weights = cnn.state_dict()

In [7]:
cnn_weights.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [8]:
cnn_weights['conv1.bias']

tensor([ 0.1128,  0.1921,  0.2129,  0.1063, -0.0257, -0.0294, -0.2667, -0.1411,
         0.2175, -0.1596, -0.1805, -0.0074, -0.2530, -0.2745,  0.2855, -0.1332])

In [9]:
cnn_weights['conv1.bias'][0]

tensor(0.1128)

In [10]:
elements_number = [p.data.nelement() for p in cnn.parameters()]
elements_number

[144, 16, 2304, 16, 25600, 64, 640, 10]

In [11]:
parameter = sum(elements_number)
print("the number of parameters is ", parameter)

the number of parameters is  28794


In [12]:
type(cnn_weights)

collections.OrderedDict

In [13]:
# weight = dict_torch2list(cnn_weights)
for key in cnn_weights.keys():
    print("{}: len {}, size {}".format(key, cnn_weights[key].data.nelement(), len(str(cnn_weights[key]))))

conv1.weight: len 144, size 1934
conv1.bias: len 16, size 160
conv2.weight: len 2304, size 5695
conv2.bias: len 16, size 160
fc1.weight: len 25600, size 433
fc1.bias: len 64, size 640
fc2.weight: len 640, size 6490
fc2.bias: len 10, size 106


## Data encryption ##

In [14]:
public_key_original, private_key_original = paillier.generate_paillier_keypair(n_length=512)

In [15]:
public_n = public_key_original.n
private_p = private_key_original.p
private_q = private_key_original.q

In [16]:
print(f"public_key.n = {public_n}")
print(f"private_key.p = {private_p}")
print(f"private_key.q = {private_q}")

public_key.n = 11951472761197597613502804581251949257187495516037693104532153031415896029056891011736635494815939010021880947653253750116332613442250805735758286361146429
private_key.p = 108475715452296325400439142182422482173260114210959121776000083324043999184437
private_key.q = 110176482463058016717527413688327112115226958910616891565463619832916994103017


In [17]:
cnn_weights["conv1.bias"].shape

torch.Size([16])

In [18]:
m = cnn_weights["conv1.bias"][0]
print(type(m), type(m.item()), m.item())
c = public_key_original.encrypt(m.item())

<class 'torch.Tensor'> <class 'float'> 0.11276400089263916


In [19]:
type(c), c.__dict__

(phe.paillier.EncryptedNumber,
 {'public_key': <PaillierPublicKey 182ca07bb7>,
  '_EncryptedNumber__ciphertext': 103621878995487562753619177248491752670660608499180588582713991724924082630510941779462832645722105947365947883843181178245421574299903278181180385127578539753485181393877405765212745894536429461828800813968376907288116859997831157414142736177655896399940263208478857648679661389110664782866127854685164195596,
  'exponent': -14,
  '_EncryptedNumber__is_obfuscated': True})

In [20]:
c_tranS = json.dumps({"c.c": c._EncryptedNumber__ciphertext, "c.exponent": c.exponent})
c_tranS, type(c_tranS)

('{"c.c": 103621878995487562753619177248491752670660608499180588582713991724924082630510941779462832645722105947365947883843181178245421574299903278181180385127578539753485181393877405765212745894536429461828800813968376907288116859997831157414142736177655896399940263208478857648679661389110664782866127854685164195596, "c.exponent": -14}',
 str)

In [26]:
len(c_tranS), len(str(m)), len(c_tranS) // len(str(m))

(337, 14, 24)

In [27]:
getsizeof(c_tranS), getsizeof(m)

(386, 72)

In [28]:
c_tranR = eval(c_tranS)
public_key = paillier.PaillierPublicKey(public_n)
private_key = paillier.PaillierPrivateKey(public_key, private_p, private_q)
c_tranR = paillier.EncryptedNumber(public_key=public_key, ciphertext=c_tranR["c.c"], exponent=c_tranR["c.exponent"])
c_tranR

In [29]:
print(f"private_key.decrypt(cR) = {private_key_original.decrypt(c_tranR)}")
print(f"pri_key.decrypt(cR) = {private_key.decrypt(c_tranR)}")
print(f"m = {m}")

private_key.decrypt(cR) = 0.11276400089263916
pri_key.decrypt(cR) = 0.11276400089263916
m = 0.11276400089263916


## Model encryption ## 

In [30]:
cnn_weights_ndarray = dict_torch2ndarray(cnn_weights)
type(cnn_weights[list(cnn_weights.keys())[0]]), type(cnn_weights_ndarray[list(cnn_weights.keys())[0]])

(torch.Tensor, numpy.ndarray)

In [31]:
for key in cnn_weights.keys():
    print("{}: shape {}".format(key, cnn_weights_ndarray[key].shape))

conv1.weight: shape (16, 1, 3, 3)
conv1.bias: shape (16,)
conv2.weight: shape (16, 16, 3, 3)
conv2.bias: shape (16,)
fc1.weight: shape (64, 400)
fc1.bias: shape (64,)
fc2.weight: shape (10, 64)
fc2.bias: shape (10,)


In [32]:
cnn_tran = dict()
for key in cnn_weights.keys():
    # if key!= "conv1.weight":
    #     continue
    item_shape = cnn_weights_ndarray[key].shape
    # print(key, item_shape)
    weight = cnn_weights_ndarray[key].flatten()
    weight_tran = np.ndarray(shape=weight.shape, dtype=object)
    for idx in range(len(weight)):
        item_c = public_key.encrypt(weight[idx].item())
        item_cdict = {"c.c": item_c._EncryptedNumber__ciphertext, "c.exponent": item_c.exponent}
        weight_tran[idx] = json.dumps(item_cdict)

    weight_tran = weight_tran.reshape(item_shape)
    # print(key, weight_tran.shape)
    cnn_tran[key] = weight_tran.tolist() # list to json

cnn_tranT = json.dumps(cnn_tran)

In [34]:
type(cnn_tranT), len(cnn_tranT),

(str, 9914017)

In [33]:
cnn_tt = json.dumps(dict_torch2list(cnn_weights))
type(cnn_weights[list(cnn_weights.keys())[0]]), type(cnn_weights_ndarray[list(cnn_weights.keys())[0]])

(torch.Tensor, numpy.ndarray)

In [35]:
len(cnn_tranT), len(cnn_tt), len(cnn_tranT) // len(cnn_tt)

(9914017, 638538, 15)

In [36]:
getsizeof(cnn_tranT), getsizeof(cnn_tt),  getsizeof(cnn_tranT) // getsizeof(cnn_tt)

(9914066, 638587, 15)

In [41]:
getsizeof(cnn_tt) / 2 ** 20 # MB

0.609004020690918

In [40]:
getsizeof(cnn_tranT) / 2 ** 20 # MB

9.454790115356445

In [39]:
filename = 'numbers.json'          #use the file extension .json
with open(filename, 'w') as file_object:  #open the file in write mode
     json.dump(cnn_tran, file_object) 

## Model decryption ##

In [30]:
public_n, private_p, private_q

(8237232322313063965353663951081499682802714993853141538751222907688929968437769505905324851924365806569664788216338294942446346850676808644739425198080881,
 77171773208882469711442311217589312963287309451585340071767599217436363073131,
 106738927716707676646701410763274229341606917387697421075118813894364044975251)

In [31]:
public_key = paillier.PaillierPublicKey(public_n)
private_key = paillier.PaillierPrivateKey(public_key, private_p, private_q)

In [32]:
cnn_tranR = eval(cnn_tranT)
cnn_tranR_ndarray = dict_list2ndarray(cnn_tranR)
type(cnn_tranR_ndarray[list(cnn_weights.keys())[0]])

numpy.ndarray

In [33]:
eval(cnn_tranR_ndarray["conv1.bias"][0])

{'c.c': 63297438159577890125118826110131942546593955719990538794282029002440899072231500532553270704245656897460902377955313139656928792353842969182000070971255480994464737069391250876559944835216198973269870739169546237830707305367468387592806512580804035326679977362786978160090212629981373660128223620872451406924,
 'c.exponent': -14}

In [34]:
cnn_rec = dict()

for key in cnn_tranR_ndarray.keys():    
    if key != "conv1.bias":
        continue
    item_shape = cnn_tranR_ndarray[key].shape
    weight_tranT = cnn_tranR_ndarray[key].flatten()
    weight_tranR = np.ndarray(shape=weight_tranT.shape, dtype=float)
    for idx in range(len(weight_tranT)):        
        item_tranR = eval(weight_tranT[idx])
        item_c = paillier.EncryptedNumber(public_key=public_key, ciphertext=item_tranR["c.c"], exponent=item_tranR["c.exponent"])
        weight_tranR[idx] = private_key.decrypt(item_c)
        print(weight_tranR[idx])
        # print(type(weight_tranR[idx]))
    weight_tranR = weight_tranR.reshape(item_shape)
    print(weight_tranR, type(weight_tranR))
    # print(strArr.shape)
    cnn_rec[key] = weight_tranR
    print(cnn_weights["conv1.bias"])

-0.1534585952758789
-0.2859039008617401
0.31541329622268677
-0.04030021280050278
-0.2303357571363449
0.09035905450582504
0.32311075925827026
-0.2328120917081833
0.2606666684150696
0.08666690438985825
-0.14129070937633514
-0.015476226806640625
0.2905166745185852
0.19128283858299255
0.027414720505475998
0.255606472492218
[-0.1534586  -0.2859039   0.3154133  -0.04030021 -0.23033576  0.09035905
  0.32311076 -0.23281209  0.26066667  0.0866669  -0.14129071 -0.01547623
  0.29051667  0.19128284  0.02741472  0.25560647] <class 'numpy.ndarray'>
tensor([-0.1535, -0.2859,  0.3154, -0.0403, -0.2303,  0.0904,  0.3231, -0.2328,
         0.2607,  0.0867, -0.1413, -0.0155,  0.2905,  0.1913,  0.0274,  0.2556])


In [35]:
cnn_rec

{'conv1.bias': array([-0.1534586 , -0.2859039 ,  0.3154133 , -0.04030021, -0.23033576,
         0.09035905,  0.32311076, -0.23281209,  0.26066667,  0.0866669 ,
        -0.14129071, -0.01547623,  0.29051667,  0.19128284,  0.02741472,
         0.25560647])}

In [47]:
np.array(cnn_rec['conv1.bias']) == cnn_weights_ndarray['conv1.bias']

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [37]:
cnn_weights['conv1.bias']

tensor([-0.1535, -0.2859,  0.3154, -0.0403, -0.2303,  0.0904,  0.3231, -0.2328,
         0.2607,  0.0867, -0.1413, -0.0155,  0.2905,  0.1913,  0.0274,  0.2556])

In [38]:
cnn_rec['conv1.bias']

array([-0.1534586 , -0.2859039 ,  0.3154133 , -0.04030021, -0.23033576,
        0.09035905,  0.32311076, -0.23281209,  0.26066667,  0.0866669 ,
       -0.14129071, -0.01547623,  0.29051667,  0.19128284,  0.02741472,
        0.25560647])

In [39]:
for key in cnn_rec.keys():
    print(key, type(cnn_rec[key]), cnn_rec[key].shape)
    print(key, type(cnn_weights[key]), cnn_weights[key].shape)

conv1.bias <class 'numpy.ndarray'> (16,)
conv1.bias <class 'torch.Tensor'> torch.Size([16])


## Model aggregation ##

## Appendix ##

In [40]:
itee = ["AA", "bb"]
json.dumps(itee)

'["AA", "bb"]'

In [41]:
a = np.array([[1, 2], [3, 4]])
it = a.shape
a.flatten(), it, a

(array([1, 2, 3, 4]),
 (2, 2),
 array([[1, 2],
        [3, 4]]))

In [42]:
a.reshape(it)

array([[1, 2],
       [3, 4]])

In [43]:
# what is in the parameters?
# how to encrypt them?
# size calculation